In [70]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

In [71]:
TOTAL = 538
retrieved = len(diffs)
with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('originally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d["codes"]) != 0 for d in diffs)
print('processed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('files affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
originally coded with `saving`: 342/499 (68.54%)
processed commits: 78/499 (15.63%)
files affected (avg): 4.42


In [72]:
# List all codes

print('codes:', len(codes))

L = max(len(c['name']) for c in codes)
for c in codes:
    name = c['name']
    n = sum(name in d['codes'] for d in diffs)
    print(f'{name}{' ' * (L+1-len(name))}{n}')

# List codes which are attached to commits but not described

listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')

codes: 48
not_relevant                           15
digitalocean                           2
aws                                    60
gcp                                    9
azure                                  6
alicloud                               1
smaller_disk                           1
cheaper_disk_type                      2
remove_loadbalancer                    1
cheaper_instance                       15
new_gen                                12
remove_dataflow                        1
reduce_subnets                         1
remove_nat                             3
add_lifecycle_rule                     3
cheaper_volume                         3
reduce_nat_gateways                    1
remove_nat_gateway                     2
remove_eks                             1
add_preemptible                        1
reduce_vpc_endpoint_subnets            1
remove_iops                            1
cheaper_redis_cache                    1
cheaper_os                             1
re